<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **SpaceX  Falcon 9 First Stage Landing Prediction**


## Interactive Visual Analytics and Dashboard



This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component<br>
TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown<br>
TASK 3: Add a Range Slider to Select Payload<br>
TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot<br>


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans.


In [28]:
!pip install pandas
!pip3 install jupyter-dash

In [9]:
# Download dataset
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-10-05 15:35:59--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-10-05 15:36:00 (85.3 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import jupyter_dash

In [2]:
jupyter_dash.infer_jupyter_proxy_config()

AttributeError: module 'jupyter_dash' has no attribute 'infer_jupyter_proxy_config'

In [3]:
jupyter_dash.infer_jupyter_proxy_config()

AttributeError: module 'jupyter_dash' has no attribute 'infer_jupyter_proxy_config'

In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [5]:
# Build AppViewer 
from jupyterlab_dash import AppViewer
viewer = AppViewer()

In [6]:
app = dash.Dash(__name__)

# Create server variable with Flask server object for use with gunicorn
server = app.server

In [7]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                 {'label':'All sites', 'value':'ALL'},
                                                 {'label':'CAFS LC-40','value':'CCAFS LC-40'},
                                                 {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                                 {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                                 {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'}
                                             ],
                                             value='ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0:'0', 100:'100'},
                                                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [8]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
                                names='Launch Site',
                                title='Total Success Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index()
        filtered_df.rename(columns={0:'class count'}, inplace=True)
        fig = px.pie(
            filtered_df
            , values='class count'
            , names= 'class'
            , title = f'Total Success Launches for Site {entered_site}'
        )    
        return fig 
    

In [9]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, entered_payload):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.scatter(
            filtered_df
            , x='Payload Mass (kg)'
            , y='class'
            , color='Booster Version Category'
            , title=f'Correlation between Payload and Success for all sites'
        )
        return fig
    else:
        # return the outcomes pichart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'Payload Mass (kg)', 'Booster Version Category','class']).size().reset_index()
        filtered_df.rename(columns={0:'class count'}, inplace=True)
        fig = px.scatter(
            filtered_df
            , x='Payload Mass (kg)'
            , y='class'
            , color='Booster Version Category'
            , title=f'Correlation between Payload and Success for the {entered_site} site'
        )
        return fig 

In [ ]:
app.run_server(mode="inline")